1. Make sure all packages are installed
pip install numpy
pip install pandas
pip install os-sys
pip install matplotlib
pip install seaborn
pip install scikit-learn

2. Collected Data from sensor py to this file 

In [1]:
import numpy as np
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn 

In [2]:
labels = ["walk", "sit", "stand", "fall"]

for l in labels:
    #read all csv files in the directory 'kaggle_data' + l, delimit by ';'
    df = pd.concat([pd.read_csv(f, sep=';') for f in glob.glob('C:\Users\siyuz\Documents\GitHub\plumshum.github.io\sensordata.py')], ignore_index = True)
    #assign 1 to df column 'Label'
    df['Label'] = 1
    # save columns 'DeviceOrientation', 'AccelerationX','AccelerationY', 'AccelerationZ', 'Label' to csv 
    # for kaggle normalized 
    df[['DeviceOrientation', 'AccelerationX','AccelerationY', 'AccelerationZ', 'Label']].to_csv('sensor' + l + '.csv', index=False)

df = pd.concat([pd.read_csv(f) for f in glob.glob('./sensor_normalized/*.csv')], ignore_index = True)
columns = ['DeviceOrientation', 'AccelerationX','AccelerationY', 'AccelerationZ', 'Label']


TypeError: glob() missing 1 required positional argument: 'pathname'

In [ ]:
# for each column in 'Accerlation', 'AccelerationY', 'AccelerationZ', and a new column that is the fft of the original column
for c in ['AccelerationX', 'AccelerationY', 'AccelerationZ']:
    df[c + '_fft'] = np.fft.fft(df[c])

In [ ]:
# plot the first 100 column 'AccelerationX' and 'AccelerationX_fft'
# Hannahs Comment: AI does not recgonize the first fft of ____ I think, so just rewrite
df[['AccelerationX_fft', 'AccelerationY_fft', 'AccelerationZ_fft']].iloc[:100].plot()
df[['AccelerationX', 'AccelerationY', 'AccelerationZ']].iloc[:100].plot()

# Versus Head (the same)

df[['AccelerationX_fft', 'AccelerationY_fft', 'AccelerationZ_fft']].head(100).plot()
df[['AccelerationX', 'AccelerationY', 'AccelerationZ']].head(100).plot()

In [ ]:
# convert column 'DeviceOreignation' to one-hot encoding
df = pd.get_dummies(df, columns=['DeviceOrientation'])
x = np.array(df.drop(['Label'], axis=1))
y = np.array(df['Label'])

print(x)
#normalize the data
import sklearn.model_selection as model_selection
x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, test_size=0.3, random_state=42)
# Hannah's Comment my AI added random_state = 42; Makes sure that the randomizness works well or something I forgot
# Hannah's Comment: the +0.j is a because the data is shown as a complex number